# DT logs prediction model

## Contents

* [Problem description](#Problem-description)
* [Dataset](#Dataset)
* [Model architecture](#Model-architecture)
* [Model training](#Model-training)
* [Inference](#Inference)
* [Metrics evaluation](#Metrics-evaluation)
* [Conclusion](#Conclusion)

## Problem description

Predict DT logs values based on other logs values and depth info.

In [1]:
%env CUDA_VISIBLE_DEVICES=1

import os
import sys

import torch
import pickle
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, "/notebooks/goryachev/petroflow")

from petroflow import WellDataset, WS
from petroflow.batchflow import Pipeline, B, V, C
from petroflow.batchflow.research import Research, PrintLogger
from petroflow.batchflow.models.torch import UNet
from petroflow.models.logs_prediction.utils import build_dataset, calc_metrics, batch_mse, moving_average_1d

env: CUDA_VISIBLE_DEVICES=1


## Dataset

A datased of 608 wells with filtered data.

In [2]:
FILTERED_DATASET_PATH = "../data/filtered/*"
filtered_dataset = WellDataset(path=FILTERED_DATASET_PATH, dirs=True)

DT values are predicted by GK, NKTD and GZ1 logs and depth info.

In [11]:
INPUTS_COL = ['GK', 'NKTD', 'GZ1', "DEPTH KM"]
TARGET_COL = ['DT']
PROPER_COL = INPUTS_COL + TARGET_COL

4 crops of length 6.4m will be sampled from each well in a batch.

In [4]:
N_CROPS = 8
CROP_SIZE = 64
REINDEXATION_STEP = 0.1
CROP_LENGTH = CROP_SIZE * REINDEXATION_STEP

Split logs by non nan segments.

In [5]:
split_pipeline = filtered_dataset >> Pipeline().drop_nans()
batch = split_pipeline.next_batch(filtered_dataset.size)
dataset = build_dataset(batch)
dataset.split(shuffle=11)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## Model architecture

UNet model is used for logs prediction (https://analysiscenter.github.io/batchflow/api/batchflow.models.torch.unet.html).

Model configuration:
* input shape - [4, 64] (3 types of logs and depth info)
* output shape - [1, 64] - (DT logs)
* the number of filters in encoder and corresponding decoder blocks - [64, 128, 256, 512, 1024]
* each encoder and decoder block has "cna cna" layout with a kernel size of 3 and a ReLU activation
* downsampling in the encoder is performed by a max pooling operation with a kernel size and a stride of 2
* upsampling in the decoder is performed by a transposed convolution with a kernel size of 4 and a stride of 2, followed by batch normalization and a ReLU activation.

Adam optimizer with default parameters is used for model training. Mean-squared error is used as a loss function.

Model configuration specification:

In [6]:
INPUTS_SIZE = len(INPUTS_COL)
TARGET_SIZE = len(TARGET_COL)

model_config = {
    'initial_block/inputs': 'inputs',
    'inputs/inputs': {'shape': [INPUTS_SIZE, CROP_SIZE]},
    'inputs/target': {'shape': [TARGET_SIZE, CROP_SIZE]},
    'head/num_classes': 1,
    'loss': 'mse',
    'device' : C('device'),
}

## Model training

The model is trained for 200 epochs with a batch size of 4.

In [7]:
res_name = 'research_stop'
def clear_previous_results(res_name):
    if os.path.exists(res_name):
        shutil.rmtree(res_name)

In [12]:
BATCH_SIZE = 32

train_template = (Pipeline()
    .add_namespace(np)
    .random_crop(CROP_LENGTH, N_CROPS)
    .update(B('inputs'), WS('logs')[INPUTS_COL].values.ravel())
    .stack(B("inputs"), save_to=B("inputs"))
    .swapaxes(B("inputs"), 1, 2, save_to=B("inputs"))
    .array(B("inputs"), dtype=np.float32, save_to=B("inputs"))
    .update(B('target'), WS('logs')[TARGET_COL].values.ravel())
    .stack(B("target"), save_to=B("target"))
    .swapaxes(B("target"), 1, 2, save_to=B("target"))
    .array(B("target"), dtype=np.float32, save_to=B("target"))
    .init_variable('loss_history')
    .init_model('dynamic', UNet, 'unet', model_config)
    .train_model('unet', B('inputs'), B('target'), fetches='loss', save_to=V('loss_history', mode='w'))
    .run_later(batch_size=BATCH_SIZE, shuffle=True, drop_last=True, n_epochs=None)
)

train_pipeline = dataset.train >> train_template

In [13]:
test_template = (Pipeline()
    .add_namespace(np)
    .crop(CROP_LENGTH, CROP_LENGTH)
    .update(B('inputs'), WS('logs')[INPUTS_COL].values.ravel())
    .stack(B("inputs"), save_to=B("inputs"))
    .swapaxes(B("inputs"), 1, 2, save_to=B("inputs"))
    .array(B("inputs"), dtype=np.float32, save_to=B("inputs"))
    .update(B('target'), WS('logs')[TARGET_COL].values.ravel())
    .stack(B("target"), save_to=B("target"))
    .swapaxes(B("target"), 1, 2, save_to=B("target"))
    .array(B("target"), dtype=np.float32, save_to=B("target"))
    .init_variable('prediction')
    .import_model('unet', train_pipeline)
    .predict_model('unet', B('inputs'), fetches='predictions', save_to=B('prediction'))
    .init_variable('loss_history')
    .batch_mse(B("target"), B("prediction"), save_to=V('loss_history', mode='w'))
    .run_later(batch_size=BATCH_SIZE, shuffle=False, drop_last=False, n_epochs=1)
)

test_pipeline = dataset.test >> test_template

In [ ]:
TEST_EXECUTE_FREQ = 5000

ITERATIONS = 60000

clear_previous_results(res_name)

research = (Research()
            .add_pipeline(train_pipeline, variables='loss_history', name='train_pipeline')
            .add_pipeline(test_pipeline, variables='loss_history', name='test_pipeline',
                          import_from='train_pipeline', run=True, execute=TEST_EXECUTE_FREQ))

research.run(n_iters=ITERATIONS, name=res_name, devices=[0], bar=True)

Research research_stop is starting...


Domain updated: 0:   4%|▍         | 2471/60000.0 [43:11<16:45:24,  1.05s/it]

In [ ]:
df = research.load_results()
train_loss = df[df['name'] == 'train_pipeline'][['iteration', 'loss_history']].values.T
test_loss = df[df['name'] == 'test_pipeline'][['iteration', 'loss_history']].values.T

fig = plt.figure(figsize=(20, 5))
plt.plot(*train_loss)
plt.plot(moving_average_1d(train_loss[1], 50), 'r')
plt.plot(*test_loss)
plt.xlabel("Epochs")
plt.ylabel("Loss value")
plt.legend(["Train loss value", "Its moving average", "Test loss value"])
plt.show()

Train loss moving average value reaches the plateau almost instantly.

Test loss value doesn't change during learning process and that's really bad.

## Inference

Inference pipeline is similar to the training one, except for one major difference:
* `random_crop` method is changed to `crop`

In [ ]:
test_template = (Pipeline()
    .add_namespace(np)
    .crop(CROP_LENGTH, CROP_LENGTH)
    .update(B('inputs'), WS('logs')[INPUTS_COL].values.ravel())
    .stack(B("inputs"), save_to=B("inputs"))
    .swapaxes(B("inputs"), 1, 2, save_to=B("inputs"))
    .array(B("inputs"), dtype=np.float32, save_to=B("inputs"))
    .update(B('target'), WS('logs')[TARGET_COL].values.ravel())
    .stack(B("target"), save_to=B("target"))
    .swapaxes(B("target"), 1, 2, save_to=B("target"))
    .array(B("target"), dtype=np.float32, save_to=B("target"))
    .init_variable('targets', default=[])
    .update(V('targets', mode='a'), B('target'))
    .init_variable('predictions', default=[])
    .import_model('unet', train_pipeline)
    .update_config({'device': torch.device('cuda: 0')})
    .predict_model('unet', B('inputs'), fetches='predictions', save_to=V('predictions', mode='a'))
    .run(batch_size=1, shuffle=False, drop_last=False, lazy=True)
)

## Metrics evaluation

Two metrics used for model evaluation:
* Mean squared error (MSE)
* Proportion of variance explained by model to data variance (R^2)

In [ ]:
test_pipeline = dataset.test >> test_template
test_pipeline.run(n_iters=1)

true = np.concatenate([target.flatten() for target in test_pipeline.v('targets')])
pred = np.concatenate([prediction.flatten() for prediction in test_pipeline.v('predictions')])

metrics = calc_metrics(true, pred)

Plot several randomly chosen predictions.

In [ ]:
BATCH_NUM = np.random.randint(len(test_pipeline.v('predictions')))
GRAPH_NUM = test_pipeline.v('predictions')[BATCH_NUM].shape[0]
PRINT_NUM = 3
for crop_num in np.random.choice(GRAPH_NUM, PRINT_NUM, replace=False):
    true = test_pipeline.v('targets')[BATCH_NUM][crop_num, 0, :]
    pred = test_pipeline.v('predictions')[BATCH_NUM][crop_num, 0,:]

    fig = plt.figure(figsize=(15, 4))
    plt.title("Crop num {}".format(crop_num))
    plt.plot(true, 'g')
    plt.plot(pred, 'r')
    plt.legend(['true', 'pred'])
    plt.show()

## Conclusion

TODO